<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-04 12:42:44
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -6.75 K (-0.05%)
Current PnL: -13.47 L (-9.38%)
CY Booked + Current PnL: -6.73 L (-4.69%)
-------------------
Total profit:  4.52 L
Total loss:  -17.99 L
-------------------
Total Booked + Current PnL: 20.25 L (17.38%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.94%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.88 L (52.69%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.7%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,HINDALCO,651.95,761.55,-19.08,H-LC,84.38,111104.0,6792.0,10744.0,0.22,6.51,9.67,16.81,27.0,0.63,0.81,23.60,X5K,ATH,METALS
5,APOLLOHOSP,6904.43,8285.00,-16.85,H-LC,85.42,113325.0,9759.0,10947.0,-0.13,9.42,9.66,20.00,30.0,0.89,0.83,24.82,X40N,BTT,HEALTHCARE
3,ADANIPORTS,1103.69,1583.00,0.58,M-LC,31.25,101054.0,22692.0,11338.0,-0.55,28.96,11.22,43.43,77.0,2.00,0.74,35.82,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,27.08,173006.0,32166.0,12318.0,0.38,22.84,7.12,31.59,89.0,2.61,1.27,50.28,XY24,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,49.36,M-SC,88.54,129924.0,-12555.0,13174.0,-2.04,-8.81,10.14,0.44,243.0,-0.95,0.95,53.35,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,142524.0,682.0,55884.0,0.80,0.48,39.21,39.88,259.0,0.01,1.05,44.44,XY24,NTT,POWER
12,BANKINDIA,116.91,190.00,-31.29,M-MC,16.67,180300.0,492.0,111912.0,-1.06,0.27,62.07,62.52,171.0,0.00,1.32,34.45,XR,NTT,BANKS
55,LALPATHLAB,2804.19,3545.00,-32.87,H-SC,26.04,154528.0,298.0,40440.0,1.92,0.19,26.17,26.42,125.0,0.01,1.13,21.86,X40N,NTT,HEALTHCARE
38,HEROMOTOCO,4311.81,6039.03,-5.24,H-MC,58.33,152184.0,1271.0,59184.0,0.87,0.84,38.89,40.06,102.0,0.02,1.12,24.48,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,11.46,49166.0,7911.0,2896.0,5.14,19.18,5.89,26.19,1.0,2.73,0.36,15.75,XY24,ATH,IT
33,GREENPANEL,375.16,537.00,217.18,M-SC,90.62,141593.0,-41485.0,120467.0,4.35,-22.66,85.08,43.14,216.0,-0.34,1.04,33.19,XY24,NTT,MISC
65,RELAXO,902.64,1176.00,-35.41,H-SC,65.62,58432.0,-57106.0,92095.0,4.32,-49.43,157.61,30.28,134.0,-0.62,0.43,14.15,X40N,NTT,FOOTWEAR
70,SATIN,199.77,274.00,-14.59,H-SC,87.50,167780.0,-28194.0,101020.0,2.74,-14.39,60.21,37.16,130.0,-0.28,1.23,25.30,XY24,NTT,FINANCE
78,SURYODAY,189.78,240.00,57.89,H-SC,12.50,111967.0,-37959.0,77638.0,2.31,-25.32,69.34,26.46,145.0,-0.49,0.82,43.44,XR,NTT,BANKS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SAMMAANCAP,170.35,326.00,-164.77,M-SC,38.54,74424.0,-27786.0,121177.0,-8.61,-27.19,162.82,91.37,206.0,-0.23,0.55,20.81,XY25,NTT,FINANCE
11,BANDHANBNK,214.25,400.00,-31.03,H-SC,68.75,206260.0,-42270.0,257742.0,-3.33,-17.01,124.96,86.70,129.0,-0.16,1.51,36.45,XY24,NTT,BANKS
30,DMART,3764.92,5201.00,-17.03,X-LC,60.42,208314.0,23833.0,46537.0,-2.24,12.92,22.34,38.14,15.0,0.51,1.53,24.89,X40N,NTT,FMCG
87,UJJIVANSFB,52.77,53.00,49.36,M-SC,88.54,129924.0,-12555.0,13174.0,-2.04,-8.81,10.14,0.44,243.0,-0.95,0.95,53.35,OX40N,NTT,BANKS
74,SIEMENS,4406.90,7969.85,-3.94,X-LC,22.92,81108.0,-29064.0,118142.0,-2.03,-26.38,145.66,80.85,14.0,-0.25,0.59,19.46,AR,ATH,ELECTRICAL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,49.36,M-SC,88.54,129924.0,-12555.0,13174.0,-2.04,-8.81,10.14,0.44,243.0,-0.95,0.95,53.35,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-30.51,M-SC,10.42,147925.0,-23216.0,23165.0,2.03,-13.57,15.66,-0.03,189.0,-1.00,1.08,6.32,OX40N,NTT,DURABLES
92,VIPIND,488.80,489.0,-213.18,H-SC,96.88,81598.0,-13718.0,13757.0,1.31,-14.39,16.86,0.04,154.0,-1.00,0.60,60.94,OX40N,NTT,MISC
46,INDIGOPNTS,1407.73,1408.0,126.56,H-SC,91.67,148676.0,-25883.0,25914.0,-0.19,-14.83,17.43,0.02,127.0,-1.00,1.09,28.92,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.31,X-MC,17.71,86800.0,-15852.0,15850.0,-0.24,-15.44,18.26,-0.00,58.0,-1.00,0.64,31.95,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,11.46,49166.0,7911.0,2896.0,5.14,19.18,5.89,26.19,1.0,2.73,0.36,15.75,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.06,X-LC,43.75,267064.0,-28870.0,122769.0,0.68,-9.76,45.97,31.73,1.0,-0.24,1.96,6.90,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.83,X-LC,67.71,291895.0,30294.0,115328.0,0.74,11.58,39.51,55.67,2.0,0.26,2.14,18.39,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-16.87,X-LC,28.12,260557.0,-9790.0,44425.0,0.56,-3.62,17.05,12.81,6.0,-0.22,1.91,8.97,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-12.34,X-LC,29.17,277758.0,12332.0,44580.0,-0.18,4.65,16.05,21.44,9.0,0.28,2.04,11.60,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,-1.39,M-LC,27.08,173006.0,32166.0,12318.0,0.38,22.84,7.12,31.59,89.0,2.61,1.27,50.28,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,0.58,M-LC,31.25,101054.0,22692.0,11338.0,-0.55,28.96,11.22,43.43,77.0,2.00,0.74,35.82,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,73.96,73245.0,38796.0,1150.0,0.26,112.62,1.57,115.97,NaN,33.74,0.54,12.00,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,50.00,231351.0,6389.0,161275.0,-1.57,2.84,69.71,74.53,185.0,0.04,1.70,34.32,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,26.70,H-SC,48.96,129591.0,4294.0,54817.0,0.19,3.43,42.30,47.18,139.0,0.08,0.95,62.13,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.54,H-SC,37.50,117370.0,15366.0,29659.0,-1.56,15.06,25.27,44.14,124.0,0.52,0.86,33.75,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,0.58,M-LC,31.25,101054.0,22692.0,11338.0,-0.55,28.96,11.22,43.43,77.0,2.00,0.74,35.82,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,27.08,173006.0,32166.0,12318.0,0.38,22.84,7.12,31.59,89.0,2.61,1.27,50.28,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,73.96,73245.0,38796.0,1150.0,0.26,112.62,1.57,115.97,NaN,33.74,0.54,12.00,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.05,M-SC,89.58,204080.0,32459.0,129611.0,-0.96,18.91,63.51,94.43,198.0,0.25,1.50,50.62,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,83.33,167152.0,16207.0,93254.0,0.95,10.74,55.79,72.51,69.0,0.17,1.23,15.14,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-48.26,H-SC,79.17,134912.0,11576.0,122190.0,-1.34,9.39,90.57,108.45,122.0,0.09,0.99,33.84,AR,ATH,MISC
54,KPIGREEN,497.21,731.79,26.70,H-SC,48.96,129591.0,4294.0,54817.0,0.19,3.43,42.30,47.18,139.0,0.08,0.95,62.13,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,6.25,17548.0,-32045.0,969.0,1.71,-64.62,5.52,-62.66,NaN,-33.07,0.13,17.68,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,95.83,67430.0,-46119.0,86169.0,0.11,-40.62,127.79,35.27,261.0,-0.54,0.49,65.04,XR,NTT,HOTELS
58,MASFIN,326.60,399.50,-18.70,H-SC,77.08,92580.0,-5400.0,27274.0,0.39,-5.51,29.46,22.32,137.0,-0.20,0.68,34.27,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,6.09,X-SC,98.96,98810.0,-2153.0,96686.0,-1.85,-2.13,97.85,93.63,64.0,-0.02,0.72,47.17,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,0.58,M-LC,31.25,101054.0,22692.0,11338.0,-0.55,28.96,11.22,43.43,77.0,2.00,0.74,35.82,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,142524.0,682.0,55884.0,0.80,0.48,39.21,39.88,259.0,0.01,1.05,44.44,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,6.09,X-SC,98.96,98810.0,-2153.0,96686.0,-1.85,-2.13,97.85,93.63,64.0,-0.02,0.72,47.17,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-2.82,H-SC,97.92,182200.0,17530.0,51563.0,-1.10,10.65,28.30,41.95,121.0,0.34,1.34,18.91,X40N,ATH,AC
92,VIPIND,488.80,489.00,-213.18,H-SC,96.88,81598.0,-13718.0,13757.0,1.31,-14.39,16.86,0.04,154.0,-1.00,0.60,60.94,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,95.83,67430.0,-46119.0,86169.0,0.11,-40.62,127.79,35.27,261.0,-0.54,0.49,65.04,XR,NTT,HOTELS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.91
1,25,41.86
2,50,70.45


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.43
LC    33.92
MC    21.59
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.51
X40      12.28
XY25     11.95
X40N     11.92
XR       10.13
OX40N     8.21
AR        8.20
MH        1.70
X5K       1.46
X200      1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.62
M-SC    15.97
H-LC    14.12
X-LC    12.12
H-MC     9.55
X-MC     7.61
M-LC     6.64
X-SC     4.39
M-MC     4.07
L-SC     2.45
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.04,-2.05,36.20
IT,12.15,-9.70,63.58
FINANCE,7.86,-12.50,53.14
BANKS,7.80,-7.18,50.91
MISC,6.65,-14.67,67.77
PAINTS,5.72,-12.00,34.07
ELECTRICAL,5.52,2.06,43.69
HEALTHCARE,4.57,-3.34,33.53
AUTO,4.34,-14.18,64.39


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2704298.0,27
XR,1076323.0,13
AR,1025168.0,10
X40,644866.0,9
X40N,538461.0,11
XY25,490435.0,8
OX40N,308957.0,11
SR,178117.0,2
MH,84316.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1969078.0,20
M-SC,1762530.0,21
H-LC,613336.0,12
H-MC,532963.0,8
X-LC,522936.0,7
M-MC,415251.0,3
X-MC,414356.0,6
X-SC,319697.0,5
M-LC,268165.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      876813.0      6
M-SC       XY24      807033.0      7
H-SC       AR        522955.0      3
M-SC       XR        333671.0      4
M-MC       XY24      303339.0      2
H-MC       XY24      299716.0      3
X-LC       X40       238097.0      2
H-SC       X40N      213757.0      4
           XR        212375.0      3
H-LC       X40N      195646.0      4
M-SC       AR        182987.0      2
           OX40N     164670.0      5
M-LC       XY24      160170.0      3
H-LC       X40       155749.0      3
X-MC       X40       148255.0      2
L-SC       XR        147807.0      2
X-SC       XR        123230.0      1
M-SC       XY25      121177.0      1
X-LC       XY25      120160.0      3
           AR        118142.0      1
M-MC       XR        111912.0      1
X-MC       XY25      104249.0      1
X-SC       SR         96686.0      1
X-MC       XY24       93815.0      1
M-LC       XR         93254.0      1
H-SC       OX40N      88361.0      3
H-MC       XY25       83026.0      1
M-SC       SR         81431.0      1
H-LC       AR         80832.0      1
M-SC       X40        71561.0      1
H-LC       X200       69262.0      1
           X5K        67889.0      2
L-SC       AR         59918.0      1
X-SC       XY24       59705.0      1
H-MC       AR         59184.0      1
L-SC       XY24       55884.0      1
H-SC       MH         54817.0      1
L-MC       XR         54074.0      1
X-MC       X40N       52187.0      1
L-LC       XY25       47082.0      1
X-LC       X40N       46537.0      1
H-LC       XY24       43958.0      1
X-SC       OX40N      40076.0      2
H-MC       X40        31204.0      1
           X40N       30334.0      1
           MH         29499.0      1
X-MC       OX40N      15850.0      1
M-LC       XY25       14741.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
